# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

In [3]:
len(CORD19_CSV['doi'])

77448

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

In [5]:
#delete Nans
doi_counted = doi.value_counts()
doi_counted

10.31729/jnma.5498              2
10.1016/j.dsx.2020.04.012       2
10.1002/jmv.25751               1
10.1093/nar/gkr1064             1
10.1016/j.ijcha.2020.100675     1
                               ..
10.1016/j.mtcomm.2020.101734    1
10.1007/978-3-030-50423-6_37    1
10.1101/2020.11.25.20234195     1
10.1111/jth.15044               1
10.1177/0956797620939054        1
Name: doi, Length: 74302, dtype: int64

In [6]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [7]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [8]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
50082,"[{'affiliation-city': 'Evanston', 'affilname':...","{'srctype': 'j', 'eid': '2-s2.0-84924617839', ..."
50083,"[{'affiliation-city': 'Jaipur', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '3',..."
50084,"[{'affiliation-city': 'Charleston', 'affilname...","{'srctype': 'j', 'prism:issueIdentifier': '6',..."
50085,"[{'affiliation-city': 'Iksan', 'affilname': 'N...","{'srctype': 'j', 'eid': '2-s2.0-85096923466', ..."


In [9]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

50087

In [ ]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < len_dois:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 50087 -> 10.1016/j.ahj.2020.10.069
Position: 50088 -> 10.1093/infdis/jit468
Position: 50089 -> 10.1101/2020.04.27.20081349
Position: 50090 -> 10.1111/febs.13553
Position: 50091 -> 10.1007/s10389-020-01295-y
Position: 50092 -> 10.1007/978-3-030-51935-3_34
Position: 50093 -> 10.1016/j.eswa.2009.06.032
Position: 50094 -> 10.1186/1742-4682-11-22
Position: 50095 -> 10.1186/1743-422x-4-20
Position: 50096 -> 10.1038/s41380-021-01019-y
Position: 50097 -> 10.1101/2020.12.12.422477
Position: 50098 -> 10.1245/s10434-020-09026-z
Position: 50099 -> 10.1007/s11936-020-00869-z
Position: 50100 -> 10.3389/fpsyg.2020.548506
Position: 50101 -> 10.1155/2020/6675206
Position: 50102 -> 10.1016/j.tvjl.2017.06.009
Position: 50103 -> 10.1186/s12931-020-01548-0
Position: 50104 -> 10.1007/s10900-020-00952-3
Position: 50105 -> 10.1186/s13063-020-04932-9
Position: 50106 -> 10.1007/s12262-020-02592-2
Position: 50107 -> 10.1164/rccm.202007-2670le
Position: 50108 -> 10.1016/j.jviromet.2020.114013
Position: 

Position: 50267 -> 10.1016/j.childyouth.2020.105783
Position: 50268 -> 10.1002/fsn3.1622
Position: 50269 -> 10.1007/s00198-020-05483-0
Position: 50270 -> 10.12688/hrbopenres.13053.2
Position: 50271 -> 10.1016/j.physa.2016.04.009
Position: 50272 -> 10.1007/s10453-014-9342-6
Position: 50273 -> 10.1101/2020.09.25.20201582
Position: 50274 -> 10.1016/j.ajic.2006.03.014
Position: 50275 -> 10.1007/978-3-030-47426-3_20
Position: 50276 -> 10.1007/s00580-012-1405-x
Position: 50277 -> 10.3758/s13428-011-0175-8
Position: 50278 -> 10.1007/s00464-020-08006-4
Position: 50279 -> 10.1101/2020.10.16.20214049
Position: 50280 -> 10.1007/s00115-020-00973-2
Position: 50281 -> 10.1186/s40249-020-00770-8
Position: 50282 -> 10.1016/j.psychres.2020.113394
Position: 50283 -> 10.1007/s11469-020-00424-8
Position: 50284 -> 10.4081/ejtm.2019.9032
Position: 50285 -> 10.1007/s11948-019-00099-y
Position: 50286 -> 10.1007/s10707-006-0015-7
Position: 50287 -> 10.1186/s13040-021-00233-2
Position: 50288 -> 10.1183/13993003

Position: 50448 -> 10.1016/j.jpubeco.2020.104238
Position: 50449 -> 10.1038/s41392-020-00376-4
Position: 50450 -> 10.18632/aging.103716
Position: 50451 -> 10.1155/2017/2978718
Position: 50452 -> 10.1101/2020.06.05.135996
Position: 50453 -> 10.1145/3411764.3445596
Position: 50454 -> 10.1016/j.jaci.2016.12.963
Position: 50455 -> 10.1155/2019/8530273
Position: 50456 -> 10.1111/gean.12241
Position: 50457 -> 10.1016/j.bja.2020.12.002
Position: 50458 -> 10.1016/j.vaccine.2013.03.055
Position: 50459 -> 10.1038/s41380-020-00936-8
Position: 50460 -> 10.1186/s12889-020-09867-z
Position: 50461 -> 10.1016/j.jep.2016.12.042
Position: 50462 -> 10.1002/hep.31472
Position: 50463 -> 10.3390/ijerph17249418
Position: 50464 -> 10.1101/2020.07.17.20156430
Position: 50465 -> 10.1101/759902
Position: 50466 -> 10.1007/s11846-020-00423-y
Position: 50467 -> 10.1007/s11192-020-03564-9
Position: 50468 -> 10.1111/sltb.12655
Position: 50469 -> 10.3201/eid2608.201525
Position: 50470 -> 10.3390/ijerph17238827
Positio

Position: 50632 -> 10.1021/acs.jcim.0c00319
Position: 50633 -> 10.1007/s00134-020-06061-y
Position: 50634 -> 10.1016/0022-1759(92)90136-h
Position: 50635 -> 10.3390/ijerph17145059
Position: 50636 -> 10.1016/j.jns.2020.117081
Position: 50637 -> 10.1016/j.chaos.2020.110295
Position: 50638 -> 10.1038/s41423-019-0270-9
Position: 50639 -> 10.1186/s12985-016-0551-1
Position: 50640 -> 10.1007/s00466-020-01881-7
Position: 50641 -> 10.3389/fpsyt.2020.559266
Position: 50642 -> 10.3390/v12101163
Position: 50643 -> 10.1002/pro.2823
Position: 50644 -> 10.18632/oncotarget.27672
Position: 50645 -> 10.1101/2020.04.21.20073262
Position: 50646 -> 10.3389/fphar.2020.588132
Position: 50647 -> 10.1093/ajcp/aqaa187
Position: 50648 -> 10.1155/2020/9254503
Position: 50649 -> 10.1016/j.vaccine.2020.12.083
Position: 50650 -> 10.1038/gt.2009.92
Position: 50651 -> 10.1101/2020.04.06.20055848
Position: 50652 -> 10.1016/j.jinf.2020.11.028
Position: 50653 -> 10.1186/s12879-017-2754-5
Position: 50654 -> 10.1016/j.vir

Position: 50814 -> 10.1101/2020.08.14.20175299
Position: 50815 -> 10.1080/07391102.2020.1837679
Position: 50816 -> 10.1186/s13756-020-00803-9
Position: 50817 -> 10.3390/genes9110560
Position: 50818 -> 10.1371/journal.pntd.0008806
Position: 50819 -> 10.1101/2020.04.17.047324
Position: 50820 -> 10.1099/vir.0.000071
Position: 50821 -> 10.1016/j.jmii.2020.04.013
Position: 50822 -> 10.1007/s40955-020-00166-z
Position: 50823 -> 10.7717/peerj.9399
Position: 50824 -> 10.1007/s10643-020-01136-3
Position: 50825 -> 10.1016/j.forpol.2020.102253
Position: 50826 -> 10.1016/j.ajem.2020.10.068
Position: 50827 -> 10.21203/rs.3.rs-73657/v1
Position: 50828 -> 10.1038/s41467-020-19248-0
Position: 50829 -> 10.3904/kjim.2019.092
Position: 50830 -> 10.1101/753806
Position: 50831 -> 10.1186/s40249-020-00737-9
Position: 50832 -> 10.1002/jdd.12499
Position: 50833 -> 10.1007/s00262-018-2201-5
Position: 50834 -> 10.1007/s10802-019-00577-8
Position: 50835 -> 10.7326/m20-1515
Position: 50836 -> 10.1016/s0022-2836(0

...

Position: 54643 -> 10.1038/3887
Position: 54644 -> 10.1016/b978-0-08-097037-0.00053-1
Position: 54645 -> 10.1016/j.arbres.2020.05.020
Position: 54646 -> 10.1371/journal.pone.0235773
Position: 54647 -> 10.1016/j.annals.2020.103041
Position: 54648 -> 10.1038/s41598-020-77172-1
Position: 54649 -> 10.1590/2175-8239-jbn-2020-s107
Position: 54650 -> 10.1007/978-1-4614-8196-6_32
Position: 54651 -> 10.1136/bmjopen-2020-039489
Position: 54652 -> 10.1007/s11262-011-0695-4
Position: 54653 -> 10.1038/s41598-018-21058-w
Position: 54654 -> 10.1002/aepp.13128
Position: 54655 -> 10.1016/j.chiabu.2020.104739
Position: 54656 -> 10.1101/2020.05.08.20095448
Position: 54657 -> 10.1186/s12959-020-00252-9
Position: 54658 -> 10.1016/j.anpedi.2020.07.019
Position: 54659 -> 10.1038/s41467-020-20235-8
Position: 54660 -> 10.1016/j.jhin.2010.09.037
Position: 54661 -> 10.1371/journal.pbio.3000869
Position: 54662 -> 10.1093/jac/dkm157
Position: 54663 -> 10.18632/oncotarget.8097
Position: 54664 -> 10.1007/s10649-020-

Position: 54824 -> 10.1016/j.jcv.2007.07.001
Position: 54825 -> 10.1007/s12035-020-02213-9
Position: 54826 -> 10.1007/s00520-020-05740-2
Position: 54827 -> 10.1038/s41407-020-0404-z
Position: 54828 -> 10.1016/j.jcv.2003.09.011
Position: 54829 -> 10.1007/s11701-020-01100-8
Position: 54830 -> 10.3389/fpubh.2020.00252
Position: 54831 -> 10.1016/j.sleep.2020.12.003
Position: 54832 -> 10.1016/j.vaccine.2019.03.046
Position: 54833 -> 10.1111/j.1365-2885.2007.00842.x
Position: 54834 -> 10.3389/fimmu.2019.00727
Position: 54835 -> 10.3390/md17060359
Position: 54836 -> 10.1016/b978-0-12-817133-2.00018-5
Position: 54837 -> 10.1101/2020.03.22.20040071
Position: 54838 -> 10.1016/j.csbj.2020.03.025
Position: 54839 -> 10.1016/j.jhin.2018.03.031
Position: 54840 -> 10.1101/2020.07.28.20164012
Position: 54841 -> 10.1007/s10111-019-00587-y
Position: 54842 -> 10.1186/s41182-020-00217-8
Position: 54843 -> 10.3390/ijerph17218017
Position: 54844 -> 10.1371/journal.pone.0095911
Position: 54845 -> 10.1016/j.je

Position: 55006 -> 10.1111/ced.14277
Position: 55007 -> 10.1098/rsif.2017.0838
Position: 55008 -> 10.5223/pghn.2021.24.1.65
Position: 55009 -> 10.1016/j.jqsrt.2020.107496
Position: 55010 -> 10.1016/j.jfms.2011.05.016
Position: 55011 -> 10.1007/s00249-009-0478-1
Position: 55012 -> 10.1038/s41598-020-62084-x
Position: 55013 -> 10.5365/wpsar.2016.7.3.009
Position: 55014 -> 10.1002/geo2.66
Position: 55015 -> 10.1007/s00705-019-04167-3
Position: 55016 -> 10.3390/s20164553
Position: 55017 -> 10.1016/j.virusres.2014.12.030
Position: 55018 -> 10.1101/2020.04.19.20071704
Position: 55019 -> 10.1007/978-3-030-50423-6_9
Position: 55020 -> 10.1007/7355_2018_58
Position: 55021 -> 10.1016/j.socscimed.2005.07.028
Position: 55022 -> 10.1186/s13052-020-00917-1
Position: 55023 -> 10.3389/fpsyt.2020.584462
Position: 55024 -> 10.1007/s10551-020-04703-1
Position: 55025 -> 10.1007/978-1-4614-3970-7_38
Position: 55026 -> 10.1016/j.orgdyn.2009.07.005
Position: 55027 -> 10.1016/j.lisr.2011.11.002
Position: 5502

Position: 55189 -> 10.1007/bf02905277
Position: 55190 -> 10.1093/jac/dkn303
Position: 55191 -> 10.3389/fendo.2020.596898
Position: 55192 -> 10.1016/j.diabres.2020.108467
Position: 55193 -> 10.1007/s40263-020-00742-4
Position: 55194 -> 10.1101/2020.03.16.20034934
Position: 55195 -> 10.1101/2020.07.20.20157743
Position: 55196 -> 10.1007/978-1-4419-1327-2_1
Position: 55197 -> 10.1016/j.ajem.2020.05.073
Position: 55198 -> 10.1007/978-3-319-95111-9_22
Position: 55199 -> 10.1186/1471-2105-4-43
Position: 55200 -> 10.1186/s13063-020-04981-0
Position: 55201 -> 10.1101/2020.05.26.20113373
Position: 55202 -> 10.1111/j.1742-6723.2004.00606.x
Position: 55203 -> 10.3892/mmr.2020.11400
Position: 55204 -> 10.1111/jors.12510
Position: 55205 -> 10.1097/wno.0000000000001038
Position: 55206 -> 10.1186/s12985-021-01492-5
Position: 55207 -> 10.1101/2020.11.05.20223289
Position: 55208 -> 10.1007/s10140-020-01808-y
Position: 55209 -> 10.1101/2020.07.03.186825
Position: 55210 -> 10.1186/1471-2334-8-117
Positio

Position: 55370 -> 10.1016/j.jdent.2020.103480
Position: 55371 -> 10.1016/j.patrec.2020.09.010
Position: 55372 -> 10.5765/jkacap.200046
Position: 55373 -> 10.1093/infdis/jix209
Position: 55374 -> 10.3390/biom10070998
Position: 55375 -> 10.1016/j.dsx.2020.12.040
Position: 55376 -> 10.1101/2021.01.20.21249279
Position: 55377 -> 10.1038/s41386-020-0750-8
Position: 55378 -> 10.1007/s00417-020-04694-9
Position: 55379 -> 10.1101/2020.12.24.424332
Position: 55380 -> 10.1186/s13756-016-0152-1
Position: 55381 -> 10.1038/s41598-021-82212-5
Position: 55382 -> 10.1007/s40123-020-00327-w
Position: 55383 -> 10.3390/ijms21218001
Position: 55384 -> 10.1007/s43683-020-00003-2
Position: 55385 -> 10.1101/2020.07.23.20160887
Position: 55386 -> 10.14202/vetworld.2018.977-985
Position: 55387 -> 10.1101/2020.08.13.20174078
Position: 55388 -> 10.1007/978-0-387-34163-7_17
Position: 55389 -> 10.1007/978-3-030-44914-8_18
Position: 55390 -> 10.1186/1471-2229-11-178
Position: 55391 -> 10.1093/ve/veaa053
Position: 

In [ ]:
# dict_new_extra_info_saver = dict()
# i = len_df_current_extra_info
# while i < 47683:
#     #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#     dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#     i = i + 1 

# dict_new_extra_info = dict_new_extra_info_saver

In [ ]:
dict_new_extra_info

In [ ]:
df_new_extra_info = pd.DataFrame(dict_new_extra_info)

In [ ]:
df_new_extra_info

In [ ]:
df_new_extra_transposed = df_new_extra_info.T
df_new_extra_transposed

In [ ]:
df_combined_extra_info = pd.concat([df_current_extra_info, df_new_extra_transposed],ignore_index=True)

In [ ]:
df_combined_extra_info.to_pickle('extra_info_CS5099.pkl')
df_combined_extra_info

In [ ]:
entry = AffiliationsFromScopusByDOI(client, '10.1086/605034')
entry

In [ ]:
print (json.dumps(entry, ensure_ascii=False, indent=4))

In [ ]:
df_test = pd.DataFrame.from_dict(entry['affiliation'], orient='columns')
df_test

In [ ]:
entry['coredata']

In [ ]:
## ScienceDirect (full-text) document example using DOI
doi_doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")
doi_doc